In [ ]:
!pip install discord.py
!pip install groq
!pip install nest_asyncio
!pip install asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 1.9 MB/s eta 0:00:00


In [ ]:
import discord
from discord.ext import commands
import os
import csv
from datetime import datetime
from groq import Groq
from google.colab import drive
from google.colab import output
import asyncio
import nest_asyncio
import json
import re
import ast

In [ ]:
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
api_key = "GROQ_API_KEY"
client = Groq(api_key=api_key)
nest_asyncio.apply()
intents = discord.Intents.default()
intents.messages = True
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

USER_DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/UserData'
history_file_path = '/content/drive/My Drive/expense_history.json'

if not os.path.exists(history_file_path):
    with open(history_file_path, 'w') as file:
        json.dump([], file)

In [ ]:
def load_history():
    with open(history_file_path, 'r') as file:
        return json.load(file)

In [ ]:
def save_history(message, result):
    # Lấy dữ liệu lịch sử cũ
    history = load_history()

    # Thêm tin nhắn người dùng và kết quả vào lịch sử
    history.append({
        "message": message,
        "result": result
    })

    # Lưu lại lịch sử vào Google Drive
    with open(history_file_path, 'w') as file:
        json.dump(history, file)


In [ ]:
def create_csv(user_id):
    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')
    header = ['Ngày', 'Mục chi', 'Số tiền', 'Mô tả']

    with open(file_path, mode='w', newline='', encoding='utf-8') as f: #w mode để nếu chưa có file thì tạo file mới
        writer = csv.writer(f)
        writer.writerow(header)
    return file_path

In [ ]:
async def send_csv_to_user(user_id, channel):
    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')
    if os.path.exists(file_path):

        with open(file_path, 'rb') as f: # rb để ở chế độ đọc nhị phân để gửi file
            await channel.send(file=discord.File(f, f'{user_id}_expenses.csv'))
    else:
        await channel.send("Bạn chưa có dữ liệu chi tiêu nào.")

In [ ]:
async def send_month_csv_to_user(user_id, channel,new_file_path):
    if os.path.exists(new_file_path):
        with open(new_file_path, 'rb') as f: # rb để ở chế độ đọc nhị phân để gửi file
            await channel.send(file=discord.File(f, f'{user_id}_expenses.csv'))
    else:
        await channel.send("Bạn chưa có dữ liệu chi tiêu nào.")

In [ ]:
def check_expense_exists(user_id, date, category, description):
    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')

    # Kiểm tra nếu file không tồn tại
    if not os.path.exists(file_path):
        return False

    # Đọc tệp CSV và kiểm tra dữ liệu
    with open(file_path, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # Bỏ qua dòng header

        for row in reader:
            # row[0] là Ngày, row[1] là Mục chi, row[2] là Số tiền, row[3] là Mô tả
            if (row[0] == date and row[3] == description):
                return True  # Nếu tìm thấy chi tiêu phù hợp

    return False  # Nếu không tìm thấy chi tiêu phù hợp


In [ ]:
def save_expense_to_csv(user_id, date, category, amount, description, new_description):
    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')

    if not os.path.exists(file_path):
        file_path = create_csv(user_id)


    with open(file_path, mode='a', newline='', encoding='utf-8') as f:# a mode để append vô cuối file mà k  ảnh hưởng các data trước
        writer = csv.writer(f)
        writer.writerow([date, category, amount, description])

In [ ]:
def update_expense(user_id, date, category, amount, description,new_description):
    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')
    if os.path.exists(file_path):
        updated_rows = []
        with open(file_path, mode='r', encoding='utf-8') as f: #r mode chi đọc data
            reader = csv.reader(f)
            header = next(reader)
            updated_rows.append(header)  #giữ lại header
            for row in reader:
                if row[0] == date and description.lower() in row[3].lower() and description !="":
                    updated_rows.append([date, category, amount, new_description])
                    print("Update successfully")
                else:
                    updated_rows.append(row)


        with open(file_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(updated_rows)

In [ ]:
def delete_expense(user_id, date, category, description, new_description):
    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')
    if os.path.exists(file_path):

        updated_rows = []
        with open(file_path, mode='r', encoding='utf-8') as f:
            reader = csv.reader(f)
            header = next(reader)
            updated_rows.append(header)

            for row in reader:
                if not (row[0] == date and description.lower() in row[3].lower() and description !=""):
                    updated_rows.append(row)


        with open(file_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(updated_rows)



In [ ]:
def get_total_expenses_by_month(user_id, date=None):

    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')

    if not os.path.exists(file_path):
        return 0  # Nếu không có dữ liệu chi tiêu, trả về 0

    total_expenses = 0

    with open(file_path, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)  # Đọc header và bỏ qua
        for row in reader:
            expense_date = row[0]  # Ngày chi tiêu
            if date:  # Nếu có tháng cụ thể
                if expense_date.startswith(date):  # Kiểm tra nếu ngày chi tiêu nằm trong tháng yêu cầu
                    total_expenses += float(row[2])  # Thêm số tiền chi tiêu vào tổng
            else:  # Nếu không có tháng cụ thể, tính tổng toàn bộ
                total_expenses += float(row[2])

    return total_expenses


In [ ]:
def get_category_expenses_by_month(user_id, date=None, category=None):

    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')

    if not os.path.exists(file_path):
        return 0  # Nếu không có dữ liệu chi tiêu, trả về 0

    total_expenses = 0

    with open(file_path, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)  # Đọc header và bỏ qua
        for row in reader:
            expense_date = row[0]  # Ngày chi tiêu
            expense_category = row[1]  # Mục chi tiêu
            if category and expense_category.lower() == category.lower():  # Nếu có danh mục
                if date:  # Nếu có tháng cụ thể
                    if expense_date.startswith(date):  # Kiểm tra nếu ngày chi tiêu nằm trong tháng yêu cầu
                        total_expenses += float(row[2])  # Thêm số tiền chi tiêu vào tổng
                else:  # Nếu không có tháng cụ thể
                    total_expenses += float(row[2])

    return total_expenses


In [ ]:
def filter_expenses_by_month(user_id, year_month):
    # Đường dẫn tới file CSV chính của người dùng
    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')

    # Kiểm tra xem file có tồn tại không
    if not os.path.exists(file_path):
        return f"File {file_path} không tồn tại."

    # Lọc các chi tiêu theo tháng
    filtered_expenses = []

    with open(file_path, mode='r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Kiểm tra nếu ngày chi tiêu có chứa năm và tháng yêu cầu
            date = row['Ngày']
            if date and date.startswith(year_month):
                filtered_expenses.append(row)

    if not filtered_expenses:
        return f"Không có chi tiêu nào cho tháng {year_month}."

    # Tạo file CSV chứa chi tiêu của tháng đó
    new_file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses_{year_month}.csv')

    header = ['Ngày', 'Mục chi', 'Số tiền', 'Mô tả']

    with open(new_file_path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(filtered_expenses)
    return new_file_path


In [ ]:
def load_latest_expense(user_id):
    file_path = os.path.join(USER_DATA_DIR, f'{user_id}_expenses.csv')

    if not os.path.exists(file_path):
        return None  # Nếu không có tệp CSV

    with open(file_path, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # Bỏ qua dòng header

        # Đọc chi tiêu mới nhất (dòng cuối cùng)
        last_row = None
        for row in reader:
            last_row = row  # Cập nhật dòng cuối cùng
        if last_row:
            return {
                'date': last_row[0],
                'category': last_row[1],
                'amount': last_row[2],
                'description': last_row[3]
            }
    return None

In [ ]:
def get_today_date():
    return datetime.today().strftime('%Y-%m-%d')

In [ ]:
def analyze_message_with_history(message,user_id):
    # Tải lịch sử phân tích
    history = load_history()

    # Chuẩn bị các thông tin lịch sử để thêm vào prompt
    history_messages = "\n".join([f"Previous analysis: {entry['message']} - Result: {entry['result']}" for entry in history])

    system_prompt = fsystem_prompt = f"""
You are an advanced expense tracking assistant.
Given a user's message, your task is to extract the following information:
- The action: add, update, delete, smonth, or total.
- The amount: the amount of money (in VND).
- The date: the date of the expense (if not mentioned, return nothing).
- The category: the category of the expense (just food, transportation, eat, gift, holiday, tool, stationery, others).
- The description: an optional description of the expense (if provided).
- If this is an update action, please include the new description.

For the 'total' action, if the message does not specify a date:
    - Assume the user is requesting the total expenses for all time.
    - If a specific month or year is mentioned (e.g. "Tổng chi tiêu tháng 2025-03"), return the total expenses for that month.
    - If a category is provided, return the total for that category within the specified month.
    - If both category and date are given, calculate the total expenses for that category in the specified month.

  The response should be a list in the following format:
    [action, amount, date, category, description, new_description]

Example 1:
"tôi chi 50k để ăn trưa ở nhà hàng" → The system should recognize this as an expense with description "ăn trưa ở nhà hàng".
respond: ['add', 50000, '', 'food', 'ăn trưa ở nhà hàng', '']
Example 2:
"không ăn trưa mà mua xe hết 10 tỉ đồng" → This should be interpreted as an **update** action, where the old description is "ăn trưa" and the new description is "mua xe" and category should be "transportation".
respond: ['update', 100000, '', 'transportation', 'ăn trưa', 'mua xe']
Example 3:
"Tổng chi tiêu tháng 2025-03" → The system should recognize this as a **total** action and return the total for March 2025.
respond: ['total', '', '2025-03', '', '', '']
Example 4:
"Tổng chi tiêu cho category food tháng 2025-03" → The system should recognize this as a **total** action and return the total for category 'food' in March 2025.
respond: ['total', '', '2025-03', 'food', '', '']
Example 5:
"Hôm nay tôi không đi bus" → The system should recognize this as a **delete** action .
respond: ['delete', '', '', '', 'bus', '']
Example 6:
"Chi tiêu trong tháng 1 năm 2020" hoặc "In ra chi tiêu trong tháng 1 năm 2020" → The system should recognize this as a **smonth** action.
respond: ['smonth', '', '2020-01', '', '', '']
Example 7:
"Nhầm 600tr mới đúng" hoặc "600tr cơ" -> The system should recognize this as an **update** action, cơ = mới đúng.
respond: ['update', 600000000, '', '', '', '']
Example 8:
"Nhầm không có mua (cái này)" hoặc "Không có mua (cái này)" -> The system should recognize this as an **delete** action.
respond: ['delete', '', '', '', '', '']
Special instructions:
- If the message contains phrases such as "I made a mistake", "correct", "update", or similar, the action should be "update".
- "I made a mistake", "mới đúng", "update", "change", "replace", or "no longer","không... mà", it should be treated as an update action.
- If the message contains phrases such as "xóa", or similar, the action should be "delete".
- Nếu người dùng bảo hôm nay hoặc không có ngày thì để trống ô date
- Bắt buộc phải có "tổng" hoặc "tiền" thì mới trả ra "total", nếu không hoặc những câu liên qua sẽ là "smonth"
"""


    user_prompt = f"USER MESSAGE: {message}"

    try:
        # Gửi yêu cầu tới LLM
        chat_completion = client.chat.completions.create(
            messages=[{
                "role": "system",
                "content": system_prompt,
            }, {
                "role": "user",
                "content": user_prompt,
            }],
            model="llama-3.3-70b-versatile",
            temperature=0.5,
            max_completion_tokens=1024,
            top_p=1,
            stop=None,
            stream=False,
        )

        # Nhận kết quả trả về từ LLM
        result = chat_completion.choices[0].message.content.strip()

        # Tách và kiểm tra kết quả
        try:

            #result = result.strip()
            matches = re.search(r"\[(.*?)\]", result)
            if matches:
              extracted_content = matches.group(0)  # This will give you the full content inside the brackets
              print(extracted_content)
            else:
              print("No square brackets found.")
            print(result)
            extracted_list = ast.literal_eval(extracted_content)
            if isinstance(extracted_list, list) and len(extracted_list) == 6:  # Kiểm tra định dạng với 6 phần tử
                action, amount, date, category, description, new_description = extracted_list
                if not description:  # Nếu không có mô tả từ người dùng
                    latest_expense = load_latest_expense(user_id)  # Lấy chi tiêu mới nhất từ lịch sử
                    if latest_expense:
                        description = latest_expense['description']
                # Kiểm tra ngày, nếu không có thì lấy ngày hôm nay
                if date == '' or date == 'None' or date == 'today' or date == None:
                    date = get_today_date()

                # Lưu kết quả phân tích vào lịch sử (cùng với tin nhắn người dùng)
                save_history(message, result)

                return [action, amount, date, category, description, new_description]

            else:
                return "Lỗi format"
        except Exception as e:
            return f"Lỗi tách data: {e}"

    except Exception as e:
        return f"Lỗi LLM: {e}"


In [ ]:
@bot.event
async def on_message(message):
    # Tránh bot tự đọc tin nhắn của bot
    if message.author == bot.user:
        return

    user_id = str(message.author.id)
    user_message = message.content
    print(user_id)
    print(user_message)

    # Phân tích tin nhắn người dùng
    llm_response = analyze_message_with_history(user_message, user_id)
    print(llm_response)

    if isinstance(llm_response, list):
        action, amount, date, category, description, new_description = llm_response
        if amount == '' or amount == None:
            amount = 0  # Nếu không có giá trị amount thì gán bằng 0
        print(amount)
        amount = float(amount)

        # Xử lý các hành động theo yêu cầu
        if 'add' in action.lower():
            save_expense_to_csv(user_id, date, category, amount, description, new_description)
            response = "Thêm thành công"
        elif 'update' in action.lower():
          # Kiểm tra nếu chi tiêu cần update có tồn tại
            if not check_expense_exists(user_id, date, category, description):
                response = "Bạn không có chi tiêu đó để cập nhật."
            else:
                update_expense(user_id, date, category, amount, description, new_description)
                response = "Cập nhật thành công"
        elif 'delete' in action.lower():
            # Kiểm tra nếu chi tiêu cần xóa có tồn tại
            if not check_expense_exists(user_id, date, category, description):
                response = "Bạn không có chi tiêu đó để xóa."
            else:
                delete_expense(user_id, date, category, description, new_description)
                response = "Xóa thành công"
        elif 'total' in action.lower():
            # Kiểm tra nếu người dùng yêu cầu tổng chi tiêu cho tháng cụ thể
            if date:
                total_expenses = get_total_expenses_by_month(user_id, date)
                response = f"Tổng chi tiêu của bạn trong tháng {date} là: {total_expenses} VND."
            else:
                total_expenses = get_total_expenses_by_month(user_id, get_today_date())
                response = f"Tổng chi tiêu của bạn là: {total_expenses} VND."

            # Nếu có category, tính tổng theo category
            if category:
                category_total = get_category_expenses_by_month(user_id, date, category)
                response = f"Tổng chi tiêu của bạn cho {category} trong tháng {date} là: {category_total} VND."
        elif 'smonth' in action.lower():
            # Lọc chi tiêu theo tháng
            await send_month_csv_to_user(user_id,message.channel,filter_expenses_by_month(user_id, date))
            os.remove(filter_expenses_by_month(user_id, date))
        else:
            response = "Tôi không thể nhận dạng yêu cầu của bạn."

        await message.channel.send(response)

        await send_csv_to_user(user_id, message.channel)

    else:
        await message.channel.send(f"Đã xảy ra lỗi khi xử lý yêu cầu của bạn: {llm_response}")


In [ ]:
async def main():
    await bot.start('BOT_TOKEN')

asyncio.run(main())

887283252153417739
60k cơ
['update', 60000, '', '', '', '']
['update', 60000, '', '', '', '']
['update', 60000, '2025-03-15', '', 'ăn bún bò', '']
60000
Update successfully


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7a7582752450>


887283252153417739
23/5/2025 ăn sáng 50k
['add', 50000, '2025-05-23', 'food', 'ăn sáng', '']
['add', 50000, '2025-05-23', 'food', 'ăn sáng', '']
['add', 50000, '2025-05-23', 'food', 'ăn sáng', '']
50000
887283252153417739
ăn sáng vào ngày hôm nay mới đúng
['update', '', '', 'food', '', 'ăn sáng']
['update', '', '', 'food', '', 'ăn sáng']
['update', '', '2025-03-15', 'food', 'ăn sáng', 'ăn sáng']
0
887283252153417739
5/6/2023 mua xe 600tr
['add', 600000000, '2023-06-05', 'transportation', 'mua xe', '']
['add', 600000000, '2023-06-05', 'transportation', 'mua xe', '']
['add', 600000000, '2023-06-05', 'transportation', 'mua xe', '']
600000000
